In [1]:
from relforte import *

def antisymmetrize_and_hermitize(T):
    # antisymmetrize the residual
    T += np.einsum("ijab->abij",T.conj()) # This is the Hermitized version (i.e., [H,A]), which should then be antisymmetrized
    temp = T.copy()
    T -= np.einsum("ijab->jiab", temp)
    T += np.einsum("ijab->jiba", temp)
    T -= np.einsum("ijab->ijba", temp)
    return T

def form_amplitudes(H1, H2, d1, d2, denom_act, gamma1, mf):
    T2 = H2[mf.hole,mf.hole,mf.part,mf.part].copy() * d2
    T2[mf.ha,mf.ha,mf.pa,mf.pa] = 0

    T1 = H1[mf.hole,mf.part].copy()
    T1 += np.einsum('xu,iuax,xu->ia', denom_act, T2[:,mf.ha,:,mf.pa],gamma1)
    T1 *= d1
    T1[mf.ha,mf.pa] = 0

    return T1, T2

def regularized_denominator(x, s):
    z = np.sqrt(s) * x
    if abs(z) <= 1e-8:
        return np.sqrt(s)*(z - z**3/2 + z**5/6)
    return (1. - np.exp(-s * x**2)) / x

In [2]:
mol = pyscf.gto.M(
    verbose = 2,
    atom = '''
H 0 0 0
F 0 1.5 0
''',
    basis = 'cc-pvdz', spin=0, charge=0, symmetry=False
)
hf = RelForte(mol, verbose=True, density_fitting=False, decontract=False)
hf.run_dhf(transform=True, debug=False, frozen=(0,0), dump_mo_coeff=False)
hf.run_casci(cas=(6,8), do_fci=False, rdm_level=3, relativistic=True, semi_canonicalize=True)
hf.run_dsrg_mrpt2(s=2.0, relativistic=True, relax=None)

fixed_args = (hf.cumulants['gamma1'], hf.cumulants['eta1'], hf.cumulants['lambda2'], hf.cumulants['lambda3'], hf)

F0 = np.diag(np.diag(hf.fock))
d2_reg = np.zeros((hf.nhole,hf.nhole,hf.npart,hf.npart),dtype='complex128')
d1_reg = np.zeros((hf.nhole,hf.npart),dtype='complex128')

d2 = np.zeros((hf.nhole,hf.nhole,hf.npart,hf.npart),dtype='complex128')
d1 = np.zeros((hf.nhole,hf.npart),dtype='complex128')

fdiag = np.diag(hf.fock)
for i in range(hf.nhole):
    for k in range(hf.npart):
        d1[i,k] = fdiag[i]-fdiag[k+hf.ncore]
        d1_reg[i,k] = regularized_denominator(fdiag[i]-fdiag[k+hf.ncore], 2.0)
        for j in range(hf.nhole):
            for l in range(hf.npart):
                d2[i,j,k,l] = fdiag[i]+fdiag[j]-fdiag[k+hf.ncore]-fdiag[l+hf.ncore]
                d2_reg[i,j,k,l] = regularized_denominator(fdiag[i]+fdiag[j]-fdiag[k+hf.ncore]-fdiag[l+hf.ncore], 2.0)

              PySCF DHF interface              
Relativistic DHF Energy:         -99.9638776 Eh
PySCF RHF time:                    0.8632138 s
-----------------------------------------------
Will now allocate 66.724 MB disk space for the AO ERI tensor!

Timing report
....integral transformation:       3.8351703 s
Total time taken:                  3.8357267 s

            Relativistic CASCI(6,8)            
Will now allocate 0.013 MB memory for the CASCI Hamiltonian!
Time taken for 1-RDM build:        0.0062275 s
Time taken for 2-RDM build:        0.0217657 s
Time taken for 3-RDM build:        0.1326966 s
E_frzc:                          -88.8825774 Eh
E_cas:                           -14.2940434 Eh
E_nuc:                             3.1750633 Eh
E_casci:                        -100.0015576 Eh
E0 (from RDM):                  -100.0015576 Eh
Ecorr:                            -0.0376800 Eh

Timing summary
... Hamil build:                    0.0129647 s
... Hamil diag:                    

In [3]:
t0 = time.time()
H0A1_1b = np.zeros((hf.nlrg,hf.nlrg), dtype='complex128')
H0A1_2b = np.zeros((hf.nlrg,hf.nlrg,hf.nlrg,hf.nlrg), dtype='complex128')
H1_T2_C2(None, H0A1_2b, F0, None, None, hf.T2_1, *fixed_args)
H1_T2_C1(H0A1_1b, None, F0, None, None, hf.T2_1, *fixed_args)
H1_T1_C1(H0A1_1b, None, F0, None, hf.T1_1, None, *fixed_args)
H0A1_2b = antisymmetrize_and_hermitize(H0A1_2b)
H0A1_1b = H0A1_1b + H0A1_1b.T.conj()

H0A1A1_1b = np.zeros((hf.nlrg,hf.nlrg), dtype='complex128')
H0A1A1_2b = np.zeros((hf.nlrg,hf.nlrg,hf.nlrg,hf.nlrg), dtype='complex128')

H1_T1_C1(H0A1A1_1b, None,      H0A1_1b, None,    hf.T1_1, None,    *fixed_args)
H1_T2_C1(H0A1A1_1b, None,      H0A1_1b, None,    None,    hf.T2_1, *fixed_args)
H2_T1_C1(H0A1A1_1b, None,      None,    H0A1_2b, hf.T1_1, None,    *fixed_args)
H2_T2_C1(H0A1A1_1b, None,      None,    H0A1_2b, None,    hf.T2_1, *fixed_args)
H1_T2_C2(None,      H0A1A1_2b, H0A1_1b, None,    None,    hf.T2_1, *fixed_args)
H2_T1_C2(None,      H0A1A1_2b, None,    H0A1_2b, hf.T1_1, None,    *fixed_args)
H2_T2_C2(None,      H0A1A1_2b, None,    H0A1_2b, None,    hf.T2_1, *fixed_args)
H0A1A1_2b = antisymmetrize_and_hermitize(H0A1A1_2b)
H0A1A1_1b = H0A1A1_1b + H0A1A1_1b.T.conj()
H0A1A1_1b = -H0A1A1_1b
H0A1A1_2b = -H0A1A1_2b

t1 = time.time()
print("Time for Hbar: ", t1-t0)

E = (1./6) * H_T_C0(None, None, H0A1A1_1b, H0A1A1_2b, hf.T1_1, hf.T2_1, *fixed_args)
print(f'DSRG-MRPT3 energy pt 1: {E:.15f}')
assert np.allclose(H0A1_2b[hf.hole, hf.hole, hf.part, hf.part], -d2*hf.T2_1)

Time for Hbar:  0.527214765548706
DSRG-MRPT3 energy pt 1: 0.003323575825788-0.000000000000000j


In [5]:
F1 = hf.fock.copy().conj()
F1[hf.core, hf.core] = 0j
F1[hf.active, hf.active] = 0j
F1[hf.virt, hf.virt] = 0j

t2_rec = hf.dhf_eri_full_asym[hf.hole,hf.hole,hf.part,hf.part]*hf.d2
t2_rec[hf.ha,hf.ha,hf.pa,hf.pa] = 0
assert np.allclose(hf.T2_1.conj(), t2_rec)

In [6]:
Htilde1_1b = H0A1_1b + 2*F1
Htilde1_2b = H0A1_2b + 2*hf.dhf_eri_full_asym.conj()

Htilde1A1_1b = np.zeros((hf.nlrg,hf.nlrg), dtype='complex128')
Htilde1A1_2b = np.zeros((hf.nlrg,hf.nlrg,hf.nlrg,hf.nlrg), dtype='complex128')

H1_T1_C1(Htilde1A1_1b, None, Htilde1_1b, None,          hf.T1_1, None,    *fixed_args)
H1_T2_C1(Htilde1A1_1b, None, Htilde1_1b, None,          None,    hf.T2_1, *fixed_args)
H2_T1_C1(Htilde1A1_1b, None, None, Htilde1_2b, hf.T1_1, None,    *fixed_args)
H2_T2_C1(Htilde1A1_1b, None, None, Htilde1_2b, None,    hf.T2_1, *fixed_args)
H1_T2_C2(None, Htilde1A1_2b, Htilde1_1b, None,          None,    hf.T2_1, *fixed_args)
H2_T1_C2(None, Htilde1A1_2b, None, Htilde1_2b, hf.T1_1, None,    *fixed_args)
H2_T2_C2(None, Htilde1A1_2b, None, Htilde1_2b, None,    hf.T2_1, *fixed_args)
Htilde1A1_2b = antisymmetrize_and_hermitize(Htilde1A1_2b)# - 0.5*H0A1A1_2b
Htilde1A1_1b = (Htilde1A1_1b + Htilde1A1_1b.T.conj())# - 0.5*H0A1A1_1b

In [7]:
T1_2, T2_2 = form_amplitudes(Htilde1A1_1b, Htilde1A1_2b, hf.d1, hf.d2, hf.denom_act, hf.cumulants['gamma1'], hf)
E = 0.5*H_T_C0(None, None, Htilde1_1b, Htilde1_2b, T1_2, T2_2, *fixed_args)
print(f'DSRG-MRPT3 energy pt 2: {E:.15f}')

DSRG-MRPT3 energy pt 2: -0.004129141542555-0.000000000000000j


In [7]:
H0A2_1b = np.zeros((hf.nlrg,hf.nlrg), dtype='complex128')
H0A2_2b = np.zeros((hf.nlrg,hf.nlrg,hf.nlrg,hf.nlrg), dtype='complex128')
H1_T2_C2(None, H0A2_2b, F0, None, None, T2_2, *fixed_args)
H1_T2_C1(H0A2_1b, None, F0, None, None, T2_2, *fixed_args)
H1_T1_C1(H0A2_1b, None, F0, None, T1_2, None, *fixed_args)
H0A2_2b = antisymmetrize_and_hermitize(H0A2_2b)
H0A2_1b = H0A2_1b + H0A2_1b.T.conj()

Hbar2_1b = H0A2_1b + Htilde1A1_1b_new
Hbar2_2b = H0A2_2b + Htilde1A1_2b_new

E = H_T_C0(None, None, Hbar2_1b, Hbar2_2b, hf.T1_1, hf.T2_1, *fixed_args)
print(f'DSRG-MRPT3 energy pt 3: {E:.15f}')

DSRG-MRPT3 energy pt 3: 0.000018082230480-0.000000000000000j
